# Data Analysis Agent

## Overview
This tutorial creates an automated data analysis system that combines AI capabilities with data visualization to make exploratory data analysis more accessible and efficient.

## Purpose
This tool automates the entire data analysis workflow by:

1. Loading and preprocessing datasets from CSV or Excel files
2. Analyzing data structure and identifying key columns
3. Generating statistical summaries and visualizations
4. Providing AI-powered insights about patterns and relationships
5. Creating interactive visualizations tailored to the data

In [ ]:
# install necessary libraries
!pip install plotly openai

### 1. Import Libraries and Setup MCP decorator class

#### Imports Section
The code begins by importing various libraries needed for data analysis, visualization, and API communication:

- Standard libraries like os , json , uuid , and functools
- Data manipulation libraries ( pandas , numpy )
- OpenAI API for AI-powered analysis
- Plotly libraries for creating interactive visualizations
- Utility modules for dates, typing, and display capabilities

#### MCP Class Definition
The MCP class implements a decorator factory pattern that adds standardized context to function calls:

1. Class Purpose : The MCP class provides a way to standardize how your agent communicates with AI models and tracks operations.

2. The tool Decorator Method :
   
   - It's a static method that returns a decorator
   - Takes parameters for task_description (what the function does) and required_format (expected response format)
   - Creates a three-level nested function structure (decorator factory → decorator → wrapper)

3. Function Argument Inspection :
   
   - Uses Python's inspect module to analyze the function signature
   - Binds actual arguments to the function parameters
   - Applies default values for any missing optional parameters

4. Context Creation :
   
   - Extracts arguments into a dictionary (removing self )
   - Formats the task description with actual argument values
   - Creates a context dictionary with agent name, task, format, and parameters

5. Function Execution :
   
   - Calls the original function with all original arguments
   - Adds the new mcp_context parameter containing the context information

In [ ]:
import os
import openai
import pandas as pd
import numpy as np
import json
import uuid
import functools
from datetime import datetime
from typing import Dict, Any, Callable, List
from IPython.display import display, HTML, Markdown

# Import Plotly libraries
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# Define the MCP decorator class
class MCP:
    """Model Context Protocol module for standardized agent communication."""
    
    @staticmethod
    def tool(
        task_description: str,
        required_format: str = "json"
    ) -> Callable:
        """
        Decorator for functions that use the Model Context Protocol.
        
        Args:
            task_description (str): Template string describing the task
            required_format (str): Expected response format
            
        Returns:
            Callable: Decorated function
        """
        def decorator(func: Callable) -> Callable:
            @functools.wraps(func)
            def wrapper(self, *args, **kwargs):
                # Get function arguments
                import inspect
                sig = inspect.signature(func)
                bound_args = sig.bind(self, *args, **kwargs)
                bound_args.apply_defaults()
                
                # Create context from arguments
                arg_dict = dict(bound_args.arguments)
                arg_dict.pop('self', None)  # Remove 'self' from context
                
                # Generate task description with arguments
                formatted_task = task_description.format(**arg_dict)
                
                # Create a context dictionary
                context = {
                    "agent": self.name if hasattr(self, 'name') else type(self).__name__,
                    "task": formatted_task,
                    "format": required_format,
                    "parameters": arg_dict
                }
                
                # Call the original function with the context
                return func(self, *args, **kwargs, mcp_context=context)
            return wrapper
        return decorator

# Create an instance of the MCP module
mcp = MCP()

### 2. Implementing the DataAnalysisAgent Class

The DataAnalysisAgent class contains several key functions that work together to provide comprehensive data analysis capabilities:

#### Core Helper Functions
1. _update_context : Maintains the conversation history by recording all interactions between the user and the agent. Each message is timestamped and can be linked to artifacts.
2. _store_artifact : Creates a repository of analysis outputs, visualizations, and recommendations with unique identifiers and timestamps.
3. _prepare_mcp_messages : Formats messages for the OpenAI API, including system instructions and recent conversation history.

#### Analysis Functions
1. analyze_dataset : Performs initial data exploration by:
   
   - Loading data from CSV files
   - Extracting column information, data types, and basic statistics
   - Identifying missing values
   - Using OpenAI to generate comprehensive insights about the dataset structure
   
2. recommend_visualizations : Suggests appropriate visualizations by:
   
   - Identifying numeric and categorical columns
   - Considering previous analysis results
   - Generating tailored visualization recommendations based on data characteristics

3. create_visualizations : Generates interactive Plotly visualizations including:
   
   - Histograms with box plots for numeric distributions
   - Bar charts for categorical data
   - Scatter plot matrices for correlation analysis
   - Individual scatter plots with trend lines
   - Box plots for examining relationships between categorical and numeric variables
   
4. generate_insights : Provides actionable recommendations by:
   
   - Synthesizing previous analysis and visualization results
   - Identifying key findings and business implications
   - Suggesting next steps and further analysis opportunities
   - Formatting output in markdown for readability

In [ ]:
class DataAnalysisAgent:
    def __init__(self, api_key):
        """
        Initialize the data analysis agent with OpenAI API
        
        Args:
            api_key (str): OpenAI API key
        """
        self.api_key = api_key
        self.model = "gpt-3.5-turbo"
        self.name = "DataAnalysisAssistant"  # Add name for MCP
        
        # Initialize MCP context
        self.context = {
            "messages": [],
            "artifacts": {},
            "metadata": {
                "version": "1.0",
                "session_id": str(uuid.uuid4())
            }
        }
    
    def _update_context(self, role, content, artifact_id=None):
        """
        Update the MCP context with new interactions
        
        Args:
            role (str): The role of the message sender (user/assistant)
            content (str): The content of the message
            artifact_id (str, optional): ID of any generated artifact
        """
        message = {
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        }
        
        if artifact_id:
            message["artifact_id"] = artifact_id
            
        self.context["messages"].append(message)
        return message

    def _store_artifact(self, artifact_id, artifact_type, content):
        """
        Store artifacts in the MCP context
        
        Args:
            artifact_id (str): Unique identifier for the artifact
            artifact_type (str): Type of artifact (analysis/visualization/recommendation)
            content (str): The artifact content
        """
        self.context["artifacts"][artifact_id] = {
            "type": artifact_type,
            "content": content,
            "created_at": datetime.now().isoformat()
        }
        return artifact_id

    def _prepare_mcp_messages(self, system_prompt, user_prompt):
        """
        Prepare messages with MCP context for API calls
        
        Args:
            system_prompt (str): The system prompt
            user_prompt (str): The user prompt
            
        Returns:
            list: Messages formatted for OpenAI API with MCP context
        """
        # Create messages list with system prompt
        messages = [{"role": "system", "content": system_prompt}]
        
        # Add context messages (limited to last 5 for efficiency)
        for msg in self.context["messages"][-5:]:
            messages.append({"role": msg["role"], "content": msg["content"]})
            
        # Add the current user prompt
        messages.append({"role": "user", "content": user_prompt})
        
        return messages
    
    @mcp.tool(
        task_description="Analyze dataset: {file_path}",
        required_format="json"
    )
    def analyze_dataset(self, file_path, mcp_context=None):
        """
        Analyze a dataset using OpenAI API with MCP
        
        Args:
            file_path (str): Path to the dataset file
            mcp_context (Dict, optional): MCP context from decorator
        
        Returns:
            str: Analysis results
        """
        # Update context with user request
        analysis_request = f"Analyze dataset at {file_path}"
        self._update_context("user", analysis_request)
        
        # Load and prepare the dataset
        try:
            df = pd.read_csv(file_path)
            # Get basic dataset info
            data_info = {
                "columns": list(df.columns),
                "shape": df.shape,
                "dtypes": {col: str(dtype) for col, dtype in df.dtypes.items()},
                "sample": df.head(5).to_dict(orient='records'),
                "summary": df.describe().to_dict(),
                "missing_values": df.isnull().sum().to_dict()
            }
            
            # Convert to JSON for the prompt
            data_info_json = json.dumps(data_info, indent=2)
            
            # Prepare MCP-formatted messages
            system_prompt = """You are a data analysis expert. Analyze the provided dataset and provide insights.
            Your analysis should include:
            1. Basic statistics and data types
            2. Distribution of key variables
            3. Potential relationships between variables
            4. Data quality issues
            5. Initial hypotheses about the data
            """
            
            user_content = f"Analyze this dataset and provide key insights:\n{data_info_json}"
            
            messages = self._prepare_mcp_messages(system_prompt, user_content)
            
            # Send request with MCP context using new OpenAI client
            client = openai.OpenAI(api_key=self.api_key)
            response = client.chat.completions.create(
                model=self.model,
                messages=messages,
                user=self.context["metadata"]["session_id"]
            )
            
            analysis = response.choices[0].message.content
            
            # Store the analysis as an artifact and update context
            artifact_id = f"analysis_{len(self.context['artifacts']) + 1}"
            self._store_artifact(artifact_id, "analysis", analysis)
            self._update_context("assistant", analysis, artifact_id)
            
            return analysis
            
        except Exception as e:
            error_msg = f"Error analyzing dataset: {str(e)}"
            self._update_context("system", error_msg)
            return error_msg
    
    @mcp.tool(
        task_description="Generate visualization recommendations for dataset: {file_path}",
        required_format="json"
    )
    def recommend_visualizations(self, file_path, analysis=None, mcp_context=None):
        """
        Recommend visualizations for a dataset using OpenAI API with MCP
        
        Args:
            file_path (str): Path to the dataset file
            analysis (str, optional): Previous analysis results
            mcp_context (Dict, optional): MCP context from decorator
        
        Returns:
            str: Visualization recommendations
        """
        # Update context with user request
        viz_request = f"Recommend visualizations for dataset at {file_path}"
        self._update_context("user", viz_request)
        
        try:
            df = pd.read_csv(file_path)
            # Get basic dataset info
            data_info = {
                "columns": list(df.columns),
                "shape": df.shape,
                "dtypes": {col: str(dtype) for col, dtype in df.dtypes.items()},
                "numeric_columns": list(df.select_dtypes(include=['number']).columns),
                "categorical_columns": list(df.select_dtypes(include=['object']).columns)
            }
            
            # Convert to JSON for the prompt
            data_info_json = json.dumps(data_info, indent=2)
            
            # Prepare MCP-formatted messages
            system_prompt = """You are a data visualization expert. Recommend appropriate visualizations for the dataset.
            For each recommendation, include:
            1. The type of visualization (e.g., bar chart, scatter plot)
            2. Which columns/variables to use
            3. Why this visualization would be insightful
            4. Any specific parameters or settings to use
            """
            
            user_content = f"Recommend visualizations for this dataset:\n{data_info_json}"
            
            # Add previous analysis if available
            if analysis:
                user_content += f"\n\nPrevious analysis:\n{analysis}"
            
            messages = self._prepare_mcp_messages(system_prompt, user_content)
            
            # Send request with MCP context using new OpenAI client
            client = openai.OpenAI(api_key=self.api_key)
            response = client.chat.completions.create(
                model=self.model,
                messages=messages,
                user=self.context["metadata"]["session_id"]
            )
            
            recommendations = response.choices[0].message.content
            
            # Store the recommendations as an artifact and update context
            artifact_id = f"viz_{len(self.context['artifacts']) + 1}"
            self._store_artifact(artifact_id, "visualization", recommendations)
            self._update_context("assistant", recommendations, artifact_id)
            
            return recommendations
            
        except Exception as e:
            error_msg = f"Error recommending visualizations: {str(e)}"
            self._update_context("system", error_msg)
            return error_msg
    
    def create_visualizations(self, file_path, recommendations=None):
        """
        Create visualizations based on recommendations using Plotly
        
        Args:
            file_path (str): Path to the dataset file
            recommendations (str, optional): Visualization recommendations
            
        Returns:
            list: List of plotly figures
        """
        # Update context with user request
        viz_request = f"Create visualizations for dataset at {file_path}"
        self._update_context("user", viz_request)
        
        try:
            df = pd.read_csv(file_path)
            figures = []
            
            # 1. Distribution of numeric columns
            numeric_cols = df.select_dtypes(include=['number']).columns
            if len(numeric_cols) > 0:
                for col in numeric_cols[:5]:  # Limit to first 5 for performance
                    fig = px.histogram(df, x=col, marginal="box", title=f'Distribution of {col}')
                    fig.update_layout(template="plotly_white")
                    figures.append(fig)
                    
                    # Store visualization in MCP context
                    artifact_id = f"viz_dist_{col}_{len(self.context['artifacts']) + 1}"
                    self._store_artifact(artifact_id, "visualization_distribution", f"Distribution of {col}")
            
            # 2. Categorical data visualization
            cat_cols = df.select_dtypes(include=['object']).columns
            if len(cat_cols) > 0:
                for col in cat_cols[:3]:  # Limit to first 3 for performance
                    counts = df[col].value_counts().reset_index()
                    counts.columns = [col, 'count']
                    fig = px.bar(
                        counts, 
                        x=col, 
                        y='count', 
                        title=f'Count of {col}',
                        color=col
                    )
                    fig.update_layout(template="plotly_white")
                    figures.append(fig)
                    
                    # Store visualization in MCP context
                    artifact_id = f"viz_cat_{col}_{len(self.context['artifacts']) + 1}"
                    self._store_artifact(artifact_id, "visualization_categorical", f"Distribution of {col}")
            
            # 3. Scatter plots for numeric columns
            if len(numeric_cols) >= 2:
                # Create scatter plot matrix for first 4 numeric columns
                fig = px.scatter_matrix(
                    df, 
                    dimensions=numeric_cols[:4],
                    title="Scatter Plot Matrix"
                )
                fig.update_layout(template="plotly_white")
                figures.append(fig)
                
                # Store visualization in MCP context
                artifact_id = f"viz_scatter_{len(self.context['artifacts']) + 1}"
                self._store_artifact(artifact_id, "visualization_scatter", "Scatter plot matrix")
                
                # Create a few individual scatter plots with trend lines
                for i in range(min(3, len(numeric_cols)-1)):
                    for j in range(i+1, min(i+2, len(numeric_cols))):
                        fig = px.scatter(
                            df, 
                            x=numeric_cols[i], 
                            y=numeric_cols[j],
                            trendline="ols",
                            title=f'{numeric_cols[j]} vs {numeric_cols[i]}'
                        )
                        fig.update_layout(template="plotly_white")
                        figures.append(fig)
                        
                        # Store visualization in MCP context
                        artifact_id = f"viz_scatter_{numeric_cols[i]}_{numeric_cols[j]}_{len(self.context['artifacts']) + 1}"
                        self._store_artifact(
                            artifact_id, 
                            "visualization_scatter", 
                            f"Scatter plot of {numeric_cols[j]} vs {numeric_cols[i]}"
                        )
            
            # 4. Box plots for numeric columns grouped by categorical
            if len(numeric_cols) > 0 and len(cat_cols) > 0:
                # Choose first categorical column and first few numeric columns
                cat_col = cat_cols[0]
                for num_col in numeric_cols[:3]:
                    fig = px.box(
                        df, 
                        x=cat_col, 
                        y=num_col,
                        title=f'{num_col} by {cat_col}',
                        color=cat_col
                    )
                    fig.update_layout(template="plotly_white")
                    figures.append(fig)
                    
                    # Store visualization in MCP context
                    artifact_id = f"viz_box_{num_col}_{cat_col}_{len(self.context['artifacts']) + 1}"
                    self._store_artifact(
                        artifact_id, 
                        "visualization_box", 
                        f"Box plot of {num_col} by {cat_col}"
                    )
            
            # Update context with visualization creation
            self._update_context("assistant", "Interactive Plotly visualizations created successfully")
            
            return figures
            
        except Exception as e:
            error_msg = f"Error creating visualizations: {str(e)}"
            self._update_context("system", error_msg)
            return []  # Return empty list instead of dict to avoid display errors
    
    @mcp.tool(
        task_description="Generate insights and recommendations for dataset: {file_path}",
        required_format="markdown"
    )
    def generate_insights(self, file_path, analysis=None, visualizations=None, mcp_context=None):
        """
        Generate insights and recommendations using OpenAI API with MCP
        
        Args:
            file_path (str): Path to the dataset file
            analysis (str, optional): Previous analysis results
            visualizations (str, optional): Visualization recommendations
            mcp_context (Dict, optional): MCP context from decorator
        
        Returns:
            str: Insights and recommendations
        """
        # Update context with user request
        insight_request = f"Generate insights and recommendations for dataset at {file_path}"
        self._update_context("user", insight_request)
        
        try:
            df = pd.read_csv(file_path)
            # Get basic dataset info for context
            data_info = {
                "columns": list(df.columns),
                "shape": df.shape,
                "summary": df.describe().to_dict()
            }
            
            # Convert to JSON for the prompt
            data_info_json = json.dumps(data_info, indent=2)
            
            # Prepare MCP-formatted messages
            system_prompt = """You are a data science consultant. Provide actionable insights and recommendations.
            Your response should include:
            1. Key findings from the data
            2. Potential business implications
            3. Recommended actions based on the data
            4. Suggestions for further analysis
            Format your response in markdown with clear sections and bullet points.
            """
            
            user_content = f"Generate insights and recommendations for this dataset:\n{data_info_json}"
            
            # Add previous artifacts if available
            if analysis:
                user_content += f"\n\nPrevious analysis:\n{analysis}"
            
            if visualizations:
                user_content += f"\n\nVisualization recommendations:\n{visualizations}"
            
            messages = self._prepare_mcp_messages(system_prompt, user_content)
            
            # Send request with MCP context using new OpenAI client
            client = openai.OpenAI(api_key=self.api_key)
            response = client.chat.completions.create(
                model=self.model,
                messages=messages,
                user=self.context["metadata"]["session_id"]
            )
            
            insights = response.choices[0].message.content
            
            # Store the insights as an artifact and update context
            artifact_id = f"insights_{len(self.context['artifacts']) + 1}"
            self._store_artifact(artifact_id, "insights", insights)
            self._update_context("assistant", insights, artifact_id)
            
            return insights
            
        except Exception as e:
            error_msg = f"Error generating insights: {str(e)}"
            self._update_context("system", error_msg)
            return error_msg

### 3. Creating the Analysis Pipeline Function

The analyze_data function serves as the main entry point for data analysis, orchestrating the entire workflow by:

1. Retrieving the OpenAI API key from environment variables
2. Initializing the DataAnalysisAgent with MCP capabilities
3. Executing a complete analysis pipeline:
   - Dataset analysis with AI-powered insights
   - Visualization recommendations based on data characteristics
   - Creation of interactive Plotly visualizations
   - Generation of actionable insights and recommendations
4. Displaying results with formatted markdown and interactive plots
5. Saving the complete analysis context to a JSON file for future reference

In [ ]:
def analyze_data(file_path):
    """
    Analyze data using the MCP-enhanced DataAnalysisAgent with Plotly visualizations
    
    Args:
        file_path (str): Path to the dataset file
        
    Returns:
        DataAnalysisAgent: Configured agent instance
    """
    # Get API key from environment variable
    api_key = os.getenv('OPENAI_API_KEY')
    
    if not api_key:
        raise ValueError("Please set OPENAI_API_KEY environment variable")
    
    # Initialize the agent with MCP
    agent = DataAnalysisAgent(api_key)
    
    print(f"🚀 Running MCP data analysis for: {file_path}")
    
    # Run the analysis pipeline with Model Context Protocol
    analysis = agent.analyze_dataset(file_path)
    print("\n📊 Dataset Analysis:")
    display(Markdown(analysis))
    
    viz_recommendations = agent.recommend_visualizations(file_path, analysis)
    print("\n📈 Visualization Recommendations:")
    display(Markdown(viz_recommendations))
    
    # Create and display visualizations
    print("\n🎨 Creating Interactive Visualizations...")
    figures = agent.create_visualizations(file_path, viz_recommendations)
    for fig in figures:
        display(fig)
    
    insights = agent.generate_insights(file_path, analysis, viz_recommendations)
    print("\n💡 Insights and Recommendations:")
    display(Markdown(insights))
    
    print("\n🧠 Context Summary:")
    print(f"Session ID: {agent.context['metadata']['session_id']}")
    print(f"Messages: {len(agent.context['messages'])}")
    print(f"Artifacts: {len(agent.context['artifacts'])}")
    
    # Save MCP context to file
    with open("data_analysis_mcp_context.json", "w") as f:
        json.dump(agent.context, f, indent=2)
    print("\n💾 MCP Context saved to data_analysis_mcp_context.json")
    
    return agent

In [ ]:
# add the path to your file 
analyze_data("c:\\Users\\Omkar\\PROJECTS\\Langchain_projects\\AgenticAI\\AgentSDK_Tutorials\\advanced_agents\\StudentsPerformance.csv")

🚀 Running MCP data analysis for: c:\Users\Omkar\PROJECTS\Langchain_projects\AgenticAI\AgentSDK_Tutorials\advanced_agents\StudentsPerformance.csv

📊 Dataset Analysis:


### Basic Statistics and Data Types:
- The dataset contains 1000 rows and 8 columns.
- Data types:
    - 5 columns are of type 'object' (gender, race/ethnicity, parental level of education, lunch, test preparation course).
    - 3 columns are of type 'int64' (math score, reading score, writing score).

### Distribution of Key Variables:
- Math Scores:
    - Mean: 66.089
    - Standard Deviation: 15.163
    - Range: 0 to 100
    - Distribution: The scores range from 0 to 100 with a mean around 66.

- Reading Scores:
    - Mean: 69.169
    - Standard Deviation: 14.600
    - Range: 17 to 100
    - Distribution: The scores range from 17 to 100 with a mean around 69.

- Writing Scores:
    - Mean: 68.054
    - Standard Deviation: 15.196
    - Range: 10 to 100
    - Distribution: The scores range from 10 to 100 with a mean around 68.

### Potential Relationships Between Variables:
- It seems logical to assume that there could be a positive correlation between math, reading, and writing scores. Further analysis such as correlation calculations can provide insights into the strength of these relationships.

- We can also explore if factors like parental level of education, lunch type, and test preparation course have any significant impact on the students' scores.

### Data Quality Issues:
- There are no missing values in any of the columns based on the provided information.
- No outliers have been identified based on the summary statistics provided.

### Initial Hypotheses about the Data:
1. Students whose parents have higher levels of education might perform better in exams.
2. Students who have completed the test preparation course may have higher scores compared to those who haven't.
3. There may be gender-based performance differences in different subjects.


📈 Visualization Recommendations:


### Recommended Visualizations for the Dataset:

1. **Box Plots for Math, Reading, and Writing Scores:**
   - **Type of Visualization:** Box Plot
   - **Variables:** Math Score, Reading Score, Writing Score
   - **Insight:** Visualize the distribution of scores in each subject and identify outliers or variations in performance.
   - **Parameters:** Plot box plots for math, reading, and writing scores side by side for easy comparison.

2. **Bar Chart for Categorical Variables:**
   - **Type of Visualization:** Bar Chart
   - **Variables:** Gender, Race/Ethnicity, Parental Level of Education, Lunch, Test Preparation Course
   - **Insight:** Compare the counts or distributions of different categories within each categorical variable.
   - **Parameters:** Create a bar chart for each categorical variable to represent the frequencies or proportions of categories.

3. **Correlation Heatmap:**
   - **Type of Visualization:** Heatmap
   - **Variables:** Math Score, Reading Score, Writing Score
   - **Insight:** Explore the correlation between math, reading, and writing scores to understand the relationships between these variables.
   - **Parameters:** Generate a correlation matrix and plot it as a heatmap for easy visualization of correlations.

4. **Grouped Bar Chart for Gender-Based Performance:**
   - **Type of Visualization:** Grouped Bar Chart
   - **Variables:** Gender and Mean Scores by Subject
   - **Insight:** Compare the average performance of male and female students in math, reading, and writing.
   - **Parameters:** Plot a grouped bar chart with gender on the x-axis and mean scores for each subject as grouped bars.

5. **Violin Plot for Parental Level of Education vs. Scores:**
   - **Type of Visualization:** Violin Plot
   - **Variables:** Parental Level of Education, Math Score, Reading Score, Writing Score
   - **Insight:** Understand the distribution of scores for different levels of parental education.
   - **Parameters:** Plot a violin plot with parental level of education on the x-axis and scores on the y-axis to visualize the spread of scores.

These visualizations will help provide a comprehensive understanding of the dataset, explore relationships between variables, and identify potential insights related to student performance.


🎨 Creating Interactive Visualizations...



💡 Insights and Recommendations:


### Key Findings from the Data:
1. The dataset comprises 1000 rows and 8 columns, including information on gender, race/ethnicity, parental level of education, lunch, test preparation course, math score, reading score, and writing score.
2. The average math score is 66.089, reading score is 69.169, and writing score is 68.054.
3. The scores have varying standard deviations, indicating the spread of scores around the mean.
4. There are no missing values in the dataset, and no outliers have been identified based on the provided statistics.

### Potential Business Implications:
1. Student performance in math, reading, and writing is around average, with room for improvement.
2. Factors like parental level of education, test preparation course completion, and meal quality could influence student performance.
3. Understanding these factors can help educational institutions tailor interventions to improve student outcomes.

### Recommended Actions Based on the Data:
1. **Performance Enhancement Programs:**
   - Develop targeted programs to improve student performance in math, reading, and writing based on identified weaknesses.
   - Offer additional support for students with scores below the mean to help them catch up.

2. **Parental Involvement Initiatives:**
   - Engage parents with lower education levels in workshops or resources to support their children's learning at home.
   - Highlight the positive impact of parental involvement on student academic success.

3. **Test Preparation Courses:**
   - Promote and expand test preparation courses to help students better prepare for exams and potentially improve their scores.
   - Analyze the effectiveness of existing courses and make adjustments based on performance outcomes.

4. **Meal Assistance Programs:**
   - Consider providing nutritional support or meal assistance for students to ensure they have access to proper nutrition, which can impact cognitive abilities and academic performance.

### Suggestions for Further Analysis:
1. **Correlation Analysis:**
   - Explore the correlations between math, reading, and writing scores to better understand how performance in one subject relates to the others.
   
2. **Impact of Lunch Quality:**
   - Investigate if there is a correlation between the type of lunch (standard/ free/reduced) and student performance to determine the influence of nutrition on academic achievement.

3. **Longitudinal Analysis:**
   - Track the performance of the same group of students over time to assess academic growth and identify trends in improvement or decline.

4. **Demographic Disparities:**
   - Analyze whether there are disparities in performance based on demographic factors like race/ethnicity and gender to address potential inequalities in educational outcomes.



🧠 Context Summary:
Session ID: a3fad581-f363-41c6-a693-072b07db8340
Messages: 8
Artifacts: 15

💾 MCP Context saved to data_analysis_mcp_context.json
